In [ ]:
import numpy as np
import pylab as pl
import enaml

with enaml.imports():
    from psi.application.io import pika
    from psi.controller.calibration import tone

%matplotlib inline

In [ ]:
io_manifest = pika.IOManifest()
audio_engine = io_manifest.find('NI_audio')
frequencies = [250, 500, 1000, 2000, 4000, 8000, 16000, 32000]

In [ ]:
mic_channel = audio_engine.get_channel('microphone_channel')
mic_channel.gain = 40

In [ ]:
result = tone.tone_sens(
    frequencies=frequencies,
    engine=audio_engine,
    ao_channel_name='speaker_1',
    ai_channel_names=['microphone_channel'],
    gain=-30,
    debug=True,
    duration=0.1,
    iti=0,
    trim=0.01,
)

In [ ]:
c = result.loc['microphone_channel', ['rms', 'thd', 'spl', 'norm_spl']]
print(c)

In [ ]:
f, axes = pl.subplots(len(frequencies), 2, 
                      sharex=True, figsize=(12, 12))

for frequency, (ax1, ax2) in zip(frequencies, axes):
    w = result.loc['microphone_channel', frequency]['waveform']
    ax1.set_title(frequency)
    ax2.set_title(frequency)
    ax1.plot(w[0])
    ax2.plot(w[1])
    ax1.axis(xmin=0, xmax=2000)